#### 14-02-2021

### Task Details and Objective:
* Returns is number one issue for any startup in Pakistan. It can kill the startup in no time. 
* In this notebook I will predict the possibility of return for a given order (for eg. by city name, book name, payment method etc.)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math  

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gufhtugu-publications-dataset-challenge/GP Orders - 2.csv
/kaggle/input/gufhtugu-publications-dataset-challenge/GP Orders - 4.csv
/kaggle/input/gufhtugu-publications-dataset-challenge/GP Orders - 5.csv


### Import data

In [2]:
data = pd.read_csv('/kaggle/input/gufhtugu-publications-dataset-challenge/GP Orders - 5.csv')

data.head()

,Order Number,Order Status,Book Name,Order Date & Time,City,Payment Method,Total items,Total weight (grams)
0,307,Completed,Bitcoin Blockchain aur Crypto Currency - FREE ...,10/3/2019 15:03,faisalabad,NaN,2,200.0
1,308,Completed,Kaggle for Begginers,10/3/2019 15:54,Rawalpindi,Cash on Delivery (COD),1,200.0
2,309,Completed,Kaggle for Begginers/Bitcoin Blockchain aur Cr...,10/3/2019 16:09,Hyderabad,Cash on Delivery (COD),2,200.0
3,310,Cancelled,Kaggle for Begginers,10/3/2019 16:42,Rawalpindi,Cash on Delivery (COD),1,200.0
4,311,Completed,Bit Coin Block Chain aur Crypto Currency بٹ کو...,10/3/2019 16:44,Karachi,Cash on Delivery (COD),1,200.0


#### For data information

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Order Number          19239 non-null  int64  
 1   Order Status          19239 non-null  object 
 2   Book Name             19237 non-null  object 
 3   Order Date & Time     19239 non-null  object 
 4   City                  19238 non-null  object 
 5   Payment Method        19229 non-null  object 
 6   Total items           19239 non-null  int64  
 7   Total weight (grams)  19239 non-null  float64
dtypes: float64(1), int64(2), object(5)
memory usage: 1.2+ MB


### Checking Null values

In [4]:
data.isnull().sum()

Order Number             0
Order Status             0
Book Name                2
Order Date & Time        0
City                     1
Payment Method          10
Total items              0
Total weight (grams)     0
dtype: int64

In [5]:
data[(data.apply(lambda x : sum(x.isnull().values), axis=1) > 0)]

,Order Number,Order Status,Book Name,Order Date & Time,City,Payment Method,Total items,Total weight (grams)
0,307,Completed,Bitcoin Blockchain aur Crypto Currency - FREE ...,10/3/2019 15:03,faisalabad,NaN,2,200.0
6,313,Completed,Bitcoin Blockchain aur Crypto Currency - FREE ...,10/3/2019 19:04,Lahore,NaN,2,200.0
12350,60457,Returned,NaN,11/14/2020 0:00,Dera Ismail Khan,BankTransfer,0,200.0
15266,65221,Completed,"Blockchain, Cryptocurrency And Bitcoin/R ka Ta...",12/17/2020 7:18,NaN,BankTransfer,6,500.0
16976,67823,Returned,NaN,1/2/2021 20:31,Karachi,EasyPaisa,0,200.0
19070,70795,Completed,Europe Ke Baankey,1/16/2021 20:45,d i khan,NaN,1,200.0
19087,70819,Completed,Safar e Hajj,1/16/2021 23:56,Islamabad,NaN,1,200.0
19134,70883,Completed,Fasana Ajaib/Safar e Hajj,1/18/2021 12:09,"Jauharabad , Distt Khushab",NaN,2,200.0
19154,70921,Completed,Dilli Gadar Se Pehle/Purane Lakhnau Ki Jhalkiy...,1/19/2021 2:33,mansehra,NaN,6,200.0
19155,70922,Completed,Ajaibaat E Farang/Europe Ke Baankey,1/19/2021 2:39,mansehra,NaN,3,200.0


### Data Dimension

In [6]:
data.shape

(19239, 8)

In [7]:
data[data['Book Name'].isnull()]

,Order Number,Order Status,Book Name,Order Date & Time,City,Payment Method,Total items,Total weight (grams)
12350,60457,Returned,NaN,11/14/2020 0:00,Dera Ismail Khan,BankTransfer,0,200.0
16976,67823,Returned,NaN,1/2/2021 20:31,Karachi,EasyPaisa,0,200.0


In [8]:
data.drop(data.index[[12350, 16976]],inplace=True)

### Splitting
#### Many rows contains multiple books in same row, so splitting them to get each book in separate rows.

In [9]:
new_df = pd.DataFrame(data['Book Name'].str.split('/').tolist(), index=data['Order Number']).stack()

new_df = new_df.reset_index([0, 'Order Number'])

new_df.columns = ['Order Number', 'Book Name']

In [10]:
new_df

,Order Number,Book Name
0,307,Bitcoin Blockchain aur Crypto Currency - FREE ...
1,308,Kaggle for Begginers
2,309,Kaggle for Begginers
3,309,Bitcoin Blockchain aur Crypto Currency - FREE ...
4,310,Kaggle for Begginers
...,...,...
33248,71420,Kaggle for Begginers
33249,71420,Artificial Intelligence
33250,71420,ایک تھا الگورتھم
33251,71420,R ka Taaruf


### Merging

In [11]:
data = pd.merge(data, new_df, on=["Order Number"])

In [12]:
data.head()

,Order Number,Order Status,Book Name_x,Order Date & Time,City,Payment Method,Total items,Total weight (grams),Book Name_y
0,307,Completed,Bitcoin Blockchain aur Crypto Currency - FREE ...,10/3/2019 15:03,faisalabad,NaN,2,200.0,Bitcoin Blockchain aur Crypto Currency - FREE ...
1,308,Completed,Kaggle for Begginers,10/3/2019 15:54,Rawalpindi,Cash on Delivery (COD),1,200.0,Kaggle for Begginers
2,309,Completed,Kaggle for Begginers/Bitcoin Blockchain aur Cr...,10/3/2019 16:09,Hyderabad,Cash on Delivery (COD),2,200.0,Kaggle for Begginers
3,309,Completed,Kaggle for Begginers/Bitcoin Blockchain aur Cr...,10/3/2019 16:09,Hyderabad,Cash on Delivery (COD),2,200.0,Bitcoin Blockchain aur Crypto Currency - FREE ...
4,310,Cancelled,Kaggle for Begginers,10/3/2019 16:42,Rawalpindi,Cash on Delivery (COD),1,200.0,Kaggle for Begginers


#### New dimension

In [13]:
data.shape

(33253, 9)

### Cleaning extra columns

In [14]:
data['Book Name'] = data['Book Name_y']
data.drop(['Book Name_x'], axis=1, inplace=True)
data.drop(['Book Name_y'], axis=1, inplace=True)

In [15]:
data.dropna(inplace=True)

### Renaming

In [16]:
data['Payment Method'].replace({'Cash on delivery': 'COD', 'Cash on Delivery (COD)': 'COD'}, inplace=True)

In [17]:
data.isnull().sum()

Order Number            0
Order Status            0
Order Date & Time       0
City                    0
Payment Method          0
Total items             0
Total weight (grams)    0
Book Name               0
dtype: int64

In [18]:
data.head()

,Order Number,Order Status,Order Date & Time,City,Payment Method,Total items,Total weight (grams),Book Name
1,308,Completed,10/3/2019 15:54,Rawalpindi,COD,1,200.0,Kaggle for Begginers
2,309,Completed,10/3/2019 16:09,Hyderabad,COD,2,200.0,Kaggle for Begginers
3,309,Completed,10/3/2019 16:09,Hyderabad,COD,2,200.0,Bitcoin Blockchain aur Crypto Currency - FREE ...
4,310,Cancelled,10/3/2019 16:42,Rawalpindi,COD,1,200.0,Kaggle for Begginers
5,311,Completed,10/3/2019 16:44,Karachi,COD,1,200.0,Bit Coin Block Chain aur Crypto Currency بٹ کو...


In [19]:
data.drop(columns = ['Order Date & Time', 'Order Number'], inplace=True)

## Machine Learning

### Importing Libraries

In [20]:
import sklearn
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model, preprocessing

### Label Encoding
#### Encoding label using sklearn preprocessing

In [21]:
le = preprocessing.LabelEncoder()

city = le.fit_transform(list(data["City"]))
payment = le.fit_transform(list(data["Payment Method"]))
book = le.fit_transform(list(data["Book Name"]))
item = le.fit_transform(list(data["Total items"]))
status = le.fit_transform(list(data["Order Status"]))

#### Adding Encoded values columns of Features in DataFrame

In [22]:
data['City_Code'] , data['PM_Code'], data['Book_Code'], data['Status_Code'] = city, payment, book, status

#### Assigning X and Y Label and Splitting Training and Testing Data Set

In [23]:
X = list(zip(city, payment, book, data['Total items'],))
Y = list(status)
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, Y, test_size = 0.2)

### KNN - K Nearest Neighbour
* Setting model classifier and neighbours
* Determing model's accuracy by using model score

In [24]:
model = KNeighborsClassifier(n_neighbors= int(math.sqrt(len(x_train))))
model.fit(x_train, y_train)
acc = model.score(x_test, y_test)
print('Model Accuracy', round(acc*100, 2),'%')

Model Accuracy 92.81 %


### Predicting the test_data, and displaying the Predicted and Actual Result

In [25]:
predicted = model.predict(x_test)
names = ["Cancelled", "Completed", "Returned"]
pred_act = {}
pred_data = {}


for x in range(len(predicted)):
    pred_act[x] = "Predicted: ", names[predicted[x]], 'Actual: ', names[y_test[x]]
    
    pred_data[x] = "City: ",data[data['City_Code'] == x_test[x][0]]['City'].values[0],'Payment Method: ', data[data['PM_Code'] == x_test[x][1]]['Payment Method'].values[0],'Book Name: ', data[data['Book_Code'] == x_test[x][2]]['Book Name'].values[0], 'Total items: ', data[data['Total items'] == x_test[x][3]]['Total items'].values[0]
           

In [26]:
data['City'] = data['City'].str.lower()
data['Payment Method'] = data['Payment Method'].str.lower()
data['Book Name'] = data['Book Name'].str.lower()

## Predicting the Probability

### Predicting the possibility, whether the given order will be Returned, Completed or Cancelled in terms of probability

In [27]:
#Checking for dummy values

cty = 'karachi'.lower()
print(f'City Name: {cty.title()}')
print('\n')
paym = 'easypaisa'.lower()
print(f'Payment Method - (COD, BankTransfer, EasyPaisa, JazzCash): {paym.title()}')
print('\n')
bk = 'kaggle for begginers'.lower()
print(f'Book Name: {bk.title()}')
print('\n')
itm = 2
print(f'Number of items: {itm}')
print('\n')

ans_city = data[data['City'] == cty]['City_Code'].values[0]
ans_pm = data[data['Payment Method'] == paym]['PM_Code'].values[0]
ans_book = data[data['Book Name'] == bk]['Book_Code'].values[0]

fin = [(ans_city, ans_pm, ans_book, itm)]
print('\n')

print(f'Based on given variables:\nCity: {cty.title()} , Payment Method: {paym.title()} , Book Name: {bk.title()} , Number of items: {itm}\n')

print(f'PROBABILITY OF ORDER RETURNED: {round(model.predict_proba(fin)[0][2], 2)*100}%')

print(f'PROBABILITY OF ORDER COMPLETED: {round(model.predict_proba(fin)[0][1], 2)*100}%')

print(f'PROBABILITY OF ORDER CANCELLED: {round(model.predict_proba(fin)[0][0], 2)*100}%')




City Name: Karachi


Payment Method - (COD, BankTransfer, EasyPaisa, JazzCash): Easypaisa


Book Name: Kaggle For Begginers


Number of items: 2




Based on given variables:
City: Karachi , Payment Method: Easypaisa , Book Name: Kaggle For Begginers , Number of items: 2

PROBABILITY OF ORDER RETURNED: 2.0%
PROBABILITY OF ORDER COMPLETED: 96.0%
PROBABILITY OF ORDER CANCELLED: 2.0%


### Above task has been performed using KNN model, with model accuracy of around 93%.
### Probability of returned, completed and cancelled order has also been determined.

### Feel free to let me know what changes can be done to make the model more accurate.